In [2]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Reshape

sys.path.insert(0, os.path.abspath(os.path.join("..", "src")))
from utils import read_filenames, prepare_datasets
from processing import process_image
from data_generator import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '/Users/michal/Documents/studia/sem4/WdUM/Galaxies_Clusterization/data/images'
#path = '/Users/julia/Desktop/semestr-4/ml/projekty/projekt-2/galaxies_data/images_gz2/images'

galaxies = read_filenames(path)

In [4]:
galaxies_train, galaxies_val, galaxies_test = prepare_datasets(galaxies, 100000)

In [7]:
batch_size = 32
img_size = 30

train_gen = ImageDataGenerator(galaxies_train, batch_size, img_size)
val_gen = ImageDataGenerator(galaxies_val, batch_size, img_size)

In [8]:
encoder = Sequential([
    Flatten(input_shape=[img_size, img_size]),
    Dense(20*img_size, activation="relu"),
    Dense(16*img_size, activation="relu"),
    Dense(12*img_size, activation="relu"),
    Dense(8*img_size, activation="relu"),
    Dense(4*img_size, activation="relu"),
    Dense(2*img_size, activation="relu"),
    Dense(img_size, activation="relu")
])

# Define the decoder
decoder = Sequential([
    Dense(2*img_size, input_shape=[img_size], activation="relu"),
    Dense(4*img_size, activation="relu"),
    Dense(8*img_size, activation="relu"),
    Dense(12*img_size, activation="relu"),
    Dense(16*img_size, activation="relu"),
    Dense(20*img_size, activation="relu"),
    Dense(img_size * img_size, activation="sigmoid"),
    Reshape([img_size, img_size])
])

autoencoder = Sequential([encoder, decoder])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [10]:
for epoch in range(10):
    print(f"Epoch {epoch + 1}/{10}")

    autoencoder.fit(
        train_gen,
        epochs=1,
        validation_data=val_gen,
    )

    train_gen.on_epoch_end()
    val_gen.on_epoch_end()

    print()

Epoch 1/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 147s 73ms/step - loss: 0.3050 - val_loss: 0.2873

Epoch 2/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 147s 74ms/step - loss: 0.2858 - val_loss: 0.2835

Epoch 3/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 151s 76ms/step - loss: 0.2840 - val_loss: 0.2835

Epoch 4/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 149s 74ms/step - loss: 0.2837 - val_loss: 0.2827

Epoch 5/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 152s 76ms/step - loss: 0.2832 - val_loss: 0.2824

Epoch 6/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 153s 76ms/step - loss: 0.2827 - val_loss: 0.2817

Epoch 7/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 153s 76ms/step - loss: 0.2822 - val_loss: 0.2813

Epoch 8/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 155s 78ms/step - loss: 0.2822 - val_loss: 0.2812

Epoch 9/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 155s 77ms/step - loss: 0.2819 - val_loss: 0.2813

Epoch 10/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 154s 77ms/step - loss: 0.2818 - val_loss: 0.2812



In [29]:
autoencoder.save(f'../models/autoencoder_{img_size}by{img_size}.keras')